<a href="https://colab.research.google.com/github/meagangoldsmith/Project/blob/main/chatbot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install requests

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np
import requests

In [ ]:
###ATTEMPT TO WEATHER API BUT DID NOT SUCCEED
def get_weather(city_name):
    api_key = 'zUt6MI4N474fzecdInv1PEYeXk4vIrdt'
    base_url = 'http://api.openweathermap.org/data/2.5/weather'
    params = {
        'q': city_name + ',US',
        'appid': api_key,
        'units': 'metric'
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather_description = data['weather'][0]['description']
        temperature = data['main']['temp']
        humidity = data['main']['humidity']
        return f"The weather in {city_name} is {weather_description}. Temperature: {temperature}°C, Humidity: {humidity}%."
    else:
        return "Can not predict weather tonight!"

In [ ]:
questions = [
    'Hi',
    'How are you?',
    'What is your favorite color?',
    'Do you have a heart?',
    'Do you like movies?',
    'Do you know what a unicorn is?'
    'What is your name?',
    'What is the time?',
    'Can you play a game?',
    'What is the Earth population?',
    'What is the weather?'
    ]

In [ ]:
answers = [
    'Hi. What is your question?',
    'I am good.',
    'I do not have a favorite color.',
    'No I am a bot.',
    'Yes I do know what a unicorn is.',
    'I am bot',
    'I do not have access to current time.',
    'No but I can chat.',
    'The population over 7 billion.',
    'Yes the weather is'
    ]

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)


In [ ]:
max_length = max(max(len(seq) for seq in sequences_questions),
                 max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding = 'post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

vocabsize = len(tokenizer.word_index) + 1

In [ ]:
embedding_dim = 256
units = 1024

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocabsize, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocabsize, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocabsize, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_6 (Embedding)     (None, None, 256)            11008     ['input_7[0][0]']             
                                                                                                  
 embedding_7 (Embedding)     (None, None, 256)            11008     ['input_8[0][0]']             
                                                                                            

In [ ]:
decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1  # Assuming 1 is the start token
# Train the model
model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=100)

Epoch 1/100
5/5 [==============================] - 3s 66ms/step - loss: 3.3404 - accuracy: 0.4364
Epoch 2/100
5/5 [==============================] - 0s 15ms/step - loss: 2.2680 - accuracy: 0.5364
Epoch 3/100
5/5 [==============================] - 0s 14ms/step - loss: 2.1034 - accuracy: 0.5636
Epoch 4/100
5/5 [==============================] - 0s 14ms/step - loss: 2.0528 - accuracy: 0.5636
Epoch 5/100
5/5 [==============================] - 0s 28ms/step - loss: 1.9818 - accuracy: 0.5636
Epoch 6/100
5/5 [==============================] - 0s 12ms/step - loss: 1.9547 - accuracy: 0.5727
Epoch 7/100
5/5 [==============================] - 0s 12ms/step - loss: 1.8997 - accuracy: 0.5818
Epoch 8/100
5/5 [==============================] - 0s 11ms/step - loss: 2.0144 - accuracy: 0.5545
Epoch 9/100
5/5 [==============================] - 0s 12ms/step - loss: 1.7838 - accuracy: 0.5818
Epoch 10/100
5/5 [==============================] - 0s 12ms/step - loss: 1.8475 - accuracy: 0.5636
Epoch 11/100
5/5 [=

In [ ]:
def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1  # start token
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:  # end token
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

In [ ]:
print("Type to chat. Ask a question? Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")

Type to chat. Ask a question? Type 'quit' to exit.
You: what is the weather>
1/1 [==============================] - 0s 22ms/step
Bot: yes the weather
You: what is the weather in new york?
1/1 [==============================] - 0s 21ms/step
Bot: yes the weather
You: quit
